# Extract Downscaled Variables from Gridded Data to Table
The goal is to generate a table with each barangay represented by a row, containing the average of all grids covered by the corresponding barangay shapefile.

In [1]:
import os, sys
from pathlib import Path
import time

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import xarray as xr
import rioxarray as rxr

from functools import partial
import matplotlib.pyplot as plt
import glob

import sys
import ast
sys.path.append("../../") 

## Define paths

In [2]:
RAW_INPUT_PATH = Path("../../data/01-raw/")
RESULTS_PATH = Path("../../data/03-results/")
FIGS_PATH = Path("../../figs/")
YEARS = np.arange(2003,2023,1)
VARIABLE = 'tave'

os.makedirs(RESULTS_PATH / 'table_extracts', exist_ok=True)

## Define excluded years based on metrics
List down outputs that have metrics below the criteria threshold. These will be excluded from the tables

In [ ]:
if VARIABLE == 'precip': 
    EXCLUDED_CITIES = ['Legazpi']
elif VARIABLE == 'tmin': 
    EXCLUDED_CITIES = ['Davao','CagayanDeOro','Legazpi', 'Muntinlupa','Navotas','Mandaluyong']
elif VARIABLE == 'tmax': 
    EXCLUDED_CITIES = ['Legazpi', 'Mandaue']
elif VARIABLE == 'tave': 
    EXCLUDED_CITIES = ['Davao','CagayanDeOro','Legazpi', 'Muntinlupa','Navotas','Mandaluyong','Legazpi', 'Mandaue']

## Read bounds

In [3]:
bounds_df = pd.read_csv(RAW_INPUT_PATH / 'domains' / 'downscaling_domains_fixed.csv')
bounds_df['full_bounds'] = bounds_df['full_bounds'].apply(ast.literal_eval)
bounds_df['focused_bounds'] = bounds_df['focused_bounds'].apply(ast.literal_eval)
bounds_df

,city,full_bounds,focused_bounds
0,Dagupan,"[120.00931049408791, 120.4821769636825, 15.907...","[120.210342, 120.450668, 15.928978, 16.138177]"
1,Palayan,"[120.8625098711998, 121.33347866342953, 15.294...","[121.042557, 121.131134, 15.495371, 15.584679]"
2,MetroManila,"[120.8470354518582, 121.22452516891933, 14.273...","[120.8670354518582, 121.20452516891933, 14.293..."
3,Legazpi,"[123.63316781038878, 123.90749473184147, 12.97...","[123.68657, 123.767166, 13.113921, 13.205771]"
4,Iloilo,"[122.41374050886832, 122.69006942567574, 10.61...","[122.477646, 122.602422, 10.676429, 10.80698]"
5,Mandaue,"[123.773858, 124.048004, 10.209113, 10.476077]","[123.899689, 123.980026, 10.297017, 10.383142]"
6,Tacloban,"[124.8437, 125.11697419235658, 11.119000306165...","[124.94133, 125.04055, 11.151456, 11.283467]"
7,Zamboanga,"[121.85371390413884, 122.48448071157127, 6.812...","[122.013817, 122.199554, 6.852464, 7.015031]"
8,CagayanDeOro,"[124.37435854096319, 124.93891405194286, 8.136...","[124.554405, 124.701691, 8.38679, 8.551884]"
9,Davao,"[125.05176056799016, 125.81579781461177, 6.868...","[125.540085, 125.666771, 7.033772, 7.156422]"


In [4]:
CITY_NAMES = bounds_df['city'].unique()
OUTPUT_CITY_NAMES = ['MetroManila' if city_name in ['Muntinlupa', 'Navotas', 'Mandaluyong'] else city_name for city_name in CITY_NAMES]

## Read shapefiles

In [5]:
brgy_gdf = gpd.read_file(RAW_INPUT_PATH / 'domains' / 'target_admin_boundaries.zip')
brgy_gdf.head()

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060..."
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON ((120.34054 16.04489, 120.34054 16.044..."
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON ((121.12250 15.58028, 121.12687 15.579..."


In [6]:
shp_city_name_lookup = {'Dagupan City':'Dagupan', 'Palayan City':'Palayan', 'Legazpi City':'Legazpi', 'Iloilo City':'Iloilo',
       'Mandaue City':'Mandaue', 'Tacloban City':'Tacloban', 'Zamboanga City':'Zamboanga',
       'Cagayan de Oro City':"CagayanDeOro", 'Davao City':'Davao', 'City of Mandaluyong':'Mandaluyong',
       'City of Navotas':'Navotas', 'City of Muntinlupa':'Muntinlupa'}
city_name_shp_lookup = {y: x for x, y in shp_city_name_lookup.items()}
city_name_shp_lookup.update({'MetroManila': ['City of Navotas','City of Mandaluyong','City of Muntinlupa']})
city_name_shp_lookup 

{'Dagupan': 'Dagupan City',
 'Palayan': 'Palayan City',
 'Legazpi': 'Legazpi City',
 'Iloilo': 'Iloilo City',
 'Mandaue': 'Mandaue City',
 'Tacloban': 'Tacloban City',
 'Zamboanga': 'Zamboanga City',
 'CagayanDeOro': 'Cagayan de Oro City',
 'Davao': 'Davao City',
 'Mandaluyong': 'City of Mandaluyong',
 'Navotas': 'City of Navotas',
 'Muntinlupa': 'City of Muntinlupa',
 'MetroManila': ['City of Navotas',
  'City of Mandaluyong',
  'City of Muntinlupa']}

## Extract per barangay

In [7]:
city_data_list = []
for city_name in OUTPUT_CITY_NAMES:
    if city_name in EXCLUDED_CITIES:
        continue
    city_shp_lookup = city_name_shp_lookup[city_name] if city_name=='MetroManila' else [city_name_shp_lookup[city_name]]
    excluded_city_shp = [city_name_shp_lookup[city] for city in EXCLUDED_CITIES]
    
    city_brgy_gdf = brgy_gdf[(brgy_gdf['ADM3_EN'].isin(city_shp_lookup))&(~brgy_gdf['ADM3_EN'].isin(excluded_city_shp))].copy()
    if len(city_brgy_gdf)==0:
        continue
    city_brgy_gdf = city_brgy_gdf.to_crs('epsg:4326')
    out_ds = rxr.open_rasterio(RESULTS_PATH / VARIABLE / f"downscaled_{VARIABLE}_{city_name.lower()}_corrected.nc")
    out_ds = out_ds.rio.write_crs('epsg:4326')
    interp_out_ds = out_ds.rio.interpolate_na() # comment out if data is needed in raw form
    brgy_data_list = []
    for i,brgy in city_brgy_gdf.iterrows():
        print(f"Processing {brgy['ADM3_EN']}: BRGY {brgy['ADM4_EN']}...", end='')
        brgy_clipped_ds = interp_out_ds.rio.clip([brgy['geometry']], all_touched=True, drop=True)
        brgy_clipped_mean_ds = brgy_clipped_ds.mean(dim=['x','y']).drop_vars(['spatial_ref'])
        brgy_clipped_mean_df = brgy_clipped_mean_ds.to_dataframe().reset_index()
        brgy_clipped_mean_df['date'] = brgy_clipped_mean_df['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
        brgy_clipped_mean_df[VARIABLE]  = brgy_clipped_mean_df[VARIABLE].round(4)
        brgy_clipped_mean_df['adm4_pcode'] = brgy['ADM4_PCODE']
        brgy_clipped_mean_df['adm3_en'] = brgy['ADM3_EN']
        brgy_clipped_mean_df['adm4_en'] = brgy['ADM4_EN']
        brgy_clipped_mean_df =  brgy_clipped_mean_df[['date','adm4_pcode','adm3_en','adm4_en', VARIABLE]]
        brgy_data_list.append(brgy_clipped_mean_df)
        print('DONE!')

    brgy_data_df = pd.concat(brgy_data_list) 
    city_data_list.append(brgy_data_df)

In [12]:
# ## Run this for tave
# ## Get mean of tmin and tmax 
# city_data_list = []
# for city_name in OUTPUT_CITY_NAMES:
#     if city_name in EXCLUDED_CITIES:
#         continue
#     city_shp_lookup = city_name_shp_lookup[city_name] if city_name=='MetroManila' else [city_name_shp_lookup[city_name]]
#     excluded_city_shp = [city_name_shp_lookup[city] for city in EXCLUDED_CITIES]
#     city_brgy_gdf = brgy_gdf[(brgy_gdf['ADM3_EN'].isin(city_shp_lookup))&(~brgy_gdf['ADM3_EN'].isin(excluded_city_shp))].copy()
#     if len(city_brgy_gdf)==0:
#         continue
#     city_brgy_gdf = city_brgy_gdf.to_crs('epsg:4326')
#     out1_ds = rxr.open_rasterio(RESULTS_PATH / 'tmin'/ f"downscaled_tmin_{city_name.lower()}_corrected.nc")
#     out1_ds = out1_ds.rio.write_crs('epsg:4326')
#     interp_out1_ds = out1_ds.rio.interpolate_na() # comment out if data is needed in raw form
#     out2_ds = rxr.open_rasterio(RESULTS_PATH / 'tmax'/ f"downscaled_tmax_{city_name.lower()}_corrected.nc")
#     out2_ds = out2_ds.rio.write_crs('epsg:4326')
#     interp_out2_ds = out2_ds.rio.interpolate_na() # comment out if data is needed in raw form
#     interp_out_ds = (interp_out1_ds + interp_out2_ds) / 2
#     interp_out_ds.name = 'tave'

#     brgy_data_list = []
#     for i,brgy in city_brgy_gdf.iterrows():
#         print(f"Processing {brgy['ADM3_EN']}: BRGY {brgy['ADM4_EN']}...", end='')
#         brgy_clipped_ds = interp_out_ds.rio.clip([brgy['geometry']], all_touched=True, drop=True)
#         brgy_clipped_mean_ds = brgy_clipped_ds.mean(dim=['x','y']).drop_vars(['spatial_ref'])
#         brgy_clipped_mean_df = brgy_clipped_mean_ds.to_dataframe().reset_index()
#         brgy_clipped_mean_df['date'] = brgy_clipped_mean_df['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
#         brgy_clipped_mean_df[VARIABLE]  = brgy_clipped_mean_df[VARIABLE].round(4)
#         brgy_clipped_mean_df['adm4_pcode'] = brgy['ADM4_PCODE']
#         brgy_clipped_mean_df['adm3_en'] = brgy['ADM3_EN']
#         brgy_clipped_mean_df['adm4_en'] = brgy['ADM4_EN']
#         brgy_clipped_mean_df =  brgy_clipped_mean_df[['date','adm4_pcode','adm3_en','adm4_en', VARIABLE]]
#         brgy_data_list.append(brgy_clipped_mean_df)
#         print('DONE!')

#     brgy_data_df = pd.concat(brgy_data_list) 
#     city_data_list.append(brgy_data_df)

Processing Dagupan City: BRGY Lomboy...DONE!
Processing Dagupan City: BRGY Tapuac...DONE!
Processing Dagupan City: BRGY Pantal...DONE!
Processing Dagupan City: BRGY Barangay I (T. Bugallon)...DONE!
Processing Dagupan City: BRGY Pogo Chico...DONE!
Processing Dagupan City: BRGY Barangay IV (Zamora)...DONE!
Processing Dagupan City: BRGY Poblacion Oeste...DONE!
Processing Dagupan City: BRGY Calmay...DONE!
Processing Dagupan City: BRGY Barangay II (Nueva)...DONE!
Processing Dagupan City: BRGY Caranglaan...DONE!
Processing Dagupan City: BRGY Bonuan Gueset...DONE!
Processing Dagupan City: BRGY Pogo Grande...DONE!
Processing Dagupan City: BRGY Tebeng...DONE!
Processing Dagupan City: BRGY Bonuan Binloc...DONE!
Processing Dagupan City: BRGY Carael...DONE!
Processing Dagupan City: BRGY Salapingao...DONE!
Processing Dagupan City: BRGY Bolosan...DONE!
Processing Dagupan City: BRGY Tambac...DONE!
Processing Dagupan City: BRGY Mangin...DONE!
Processing Dagupan City: BRGY Mamalingling...DONE!
Processi

In [13]:
all_data_df = pd.concat(city_data_list) 
all_data_df

,date,adm4_pcode,adm3_en,adm4_en,tave
0,2003-01-01,PH015518016,Dagupan City,Lomboy,27.2730
1,2003-01-02,PH015518016,Dagupan City,Lomboy,25.7941
2,2003-01-03,PH015518016,Dagupan City,Lomboy,24.6129
3,2003-01-04,PH015518016,Dagupan City,Lomboy,24.7932
4,2003-01-05,PH015518016,Dagupan City,Lomboy,25.0476
...,...,...,...,...,...
7300,2022-12-27,PH097332103,Zamboanga City,Victoria,28.4305
7301,2022-12-28,PH097332103,Zamboanga City,Victoria,27.8313
7302,2022-12-29,PH097332103,Zamboanga City,Victoria,28.2872
7303,2022-12-30,PH097332103,Zamboanga City,Victoria,27.2073


In [16]:
all_data_df['adm3_en'].unique()

array(['Dagupan City', 'Palayan City', 'Iloilo City', 'Tacloban City',
       'Zamboanga City'], dtype=object)

In [15]:
all_data_df = all_data_df[~all_data_df['adm3_en'].isin(excluded_city_shp)]

In [18]:
all_data_df[['date','adm4_pcode',VARIABLE]].to_csv(RESULTS_PATH /'table_extracts'/ f"climate_atmosphere_downscaled_{VARIABLE}.csv", index=False)

In [ ]:
# # check plot time series
# data_df = data_pr_df[data_pr_df['adm4_pcode']=='PH015518016'].copy()
# data_df = data_df.set_index(pd.to_datetime(data_df['date']))
# data_df['pr'].iloc[:720].plot()

In [ ]:
# # check if brgy is within data
# fig,ax = plt.subplots()
# out_ds.isel(time=0).plot(ax=ax)
# city_brgy_gdf.iloc[[0]].boundary.plot(color='black', ax=ax)